# PyCoBi Bifurcation with Auto-07p 

In [17]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd
import os

In [2]:
directory_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis"

BandPower_x = np.load(os.path.join(directory_path, "BandPower_x.npy"), allow_pickle=True)
FastFourierTransform_x = np.load(os.path.join(directory_path, "FastFourierTransform_x.npy"), allow_pickle=True)
LombScarglePeriodogram_x = np.load(os.path.join(directory_path, "LombScarglePeriodogram_x.npy"), allow_pickle=True)
PeakFrequencies_x = np.load(os.path.join(directory_path, "PeakFrequencies_x.npy"), allow_pickle=True)
ShortTimeFourier_x = np.load(os.path.join(directory_path, "ShortTimeFourier_x.npy"), allow_pickle=True)
SpectralCentroids_x = np.load(os.path.join(directory_path, "SpectralCentroids_x.npy"), allow_pickle=True)
SpectralEdgeDensities_x = np.load(os.path.join(directory_path, "SpectralEdgeDensities_x.npy"), allow_pickle=True)
spectral_entropy_x = np.load(os.path.join(directory_path, "spectral_entropy_x.npy"), allow_pickle=True)
wavelet_x = np.load(os.path.join(directory_path, "wavelet_x.npy"), allow_pickle=True)
welchs_x = np.load(os.path.join(directory_path, "welchs_x.npy"), allow_pickle=True)

In [3]:
# Define the file paths
base_dir = '/home/vincent/AAA_projects/MVCS/Neuroscience/DataFrames/' 
eeg_df_path = base_dir + 'eeg_df.csv'

# load data
eeg_df = pd.read_csv(eeg_df_path)

print(eeg_df.head())

            Fp1           Fpz           Fp2           F7         F3  \
0 -21295.988649 -20109.716727 -24153.383752  3189.340060 -45.189275   
1 -21303.747077 -20120.746154 -24163.864012  3178.880909 -56.702035   
2 -21315.466571 -20130.126577 -24171.944343  3164.903807 -69.465350   
3 -21317.809594 -20131.044726 -24174.790986  3159.478572 -73.214591   
4 -21325.798142 -20137.522181 -24179.985166  3144.934679 -84.871628   

            Fz          F4           F8          FC5          FC1  ...  \
0 -8525.066680 -642.128590  3487.913621  6324.956639  6503.012177  ...   
1 -8532.499649 -651.966372  3477.011771  6315.078704  6496.522520  ...   
2 -8544.315275 -663.772856  3463.194795  6302.391524  6483.178723  ...   
3 -8545.873916 -666.109249  3457.870782  6297.212341  6481.970244  ...   
4 -8551.164448 -671.761501  3450.466406  6283.925509  6477.045614  ...   

            P7           P3          Pz           P4           P8  \
0  3374.048029 -3617.197964 -611.584742 -1667.222644  7523.

In [ ]:
Wilson-Cowan model

In [8]:
# Compute statistics for initial conditions
E_init = np.mean(eeg_df)
I_init = np.std(eeg_df)

print(f"Initial condition for E(0): {E_init}")
print(f"Initial condition for I(0): {I_init}")

/home/vincent/miniconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Initial condition for E(0): Fp1    -2.019481e+04
Fpz    -1.837152e+04
Fp2    -1.944451e+04
F7      1.842709e+03
F3     -2.443234e+02
Fz     -8.513399e+03
F4     -4.225759e+02
F8      4.559767e+03
FC5     6.697618e+03
FC1     7.492128e+03
FC2    -2.485212e+03
FC6     1.135458e+04
M1      2.651776e+02
T7     -7.467175e+03
C3     -1.408262e+03
Cz      3.137791e+03
C4      6.958163e+03
T8      2.392510e+03
M2     -8.349746e+03
CP5     1.576696e+03
CP1    -6.745014e+03
CP2     1.450383e+03
CP6     5.722390e+03
P7      4.161197e+03
P3     -3.961754e+03
Pz     -3.921798e+02
P4     -2.641259e+03
P8      9.034929e+03
POz    -1.156767e+04
O1     -1.129762e+04
Oz     -1.384648e+03
O2      3.931208e+03
Time    2.113894e+06
dtype: float64
Initial condition for I(0): Fp1     1.738809e+03
Fpz     1.381171e+03
Fp2     2.896979e+03
F7      5.748693e+03
F3      2.074885e+03
Fz      1.532398e+03
F4      5.396281e+02
F8      1.216357e+03
FC5     4.303830e+03
FC1     1.645656e+03
FC2     5.178472e+02
FC6  

In [ ]:
Assumptive Mapping

In [6]:
def normalize(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    
    # Handle division by zero and NaN values
    if arr_max - arr_min == 0:
        return np.zeros_like(arr)  # Return zeros array
    else:
        return (arr - arr_min) / (arr_max - arr_min)

# Load the data for BandPower_x
with open(os.path.join('/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis', "BandPower_x.npy"), 'rb') as f:
    band_power_dict = np.load(f, allow_pickle=True).item()

# Normalize each band power entry in the dictionary
for key in band_power_dict:
    band_power_dict[key] = normalize(band_power_dict[key])


In [10]:
# Load the data function
def load_data(file_name):
    with open(os.path.join('/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis', file_name), 'rb') as f:
        data = np.load(f, allow_pickle=True)
        if data.shape == ():  # check if it is a 0-dimensional array (which might contain a dictionary)
            return data.item()
        else:
            return {"data_array": data}

# Normalize function for dictionary
def normalize_dict(d):
    for key in d:
        d[key] = normalize(d[key])
    return d

# Normalize function for numpy array
def normalize(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    if arr_max - arr_min == 0:
        return np.zeros_like(arr)
    else:
        return (arr - arr_min) / (arr_max - arr_min)

# Load and normalize data
BandPower_x = normalize_dict(load_data("BandPower_x.npy"))
FastFourierTransform_x = normalize_dict(load_data("FastFourierTransform_x.npy"))
PeakFrequencies_x = normalize_dict(load_data("PeakFrequencies_x.npy"))
SpectralCentroids_x = normalize_dict(load_data("SpectralCentroids_x.npy"))
SpectralEdgeDensities_x = normalize_dict(load_data("SpectralEdgeDensities_x.npy"))
spectral_entropy_x = normalize_dict(load_data("spectral_entropy_x.npy"))
welchs_x = normalize_dict(load_data("welchs_x.npy"))

# Use normalized values to derive E_init and I_init

# Assuming these measures indicate excitatory activity
E_measure = (
    np.mean(list(BandPower_x.values())) +
    np.mean(list(FastFourierTransform_x.values())) +
    np.mean(list(PeakFrequencies_x.values())) +
    np.mean(list(SpectralCentroids_x.values())) +
    np.mean(list(SpectralEdgeDensities_x.values())) -
    np.mean(list(spectral_entropy_x.values())) +
    np.mean(list(welchs_x.values()))
) / 7

# Assuming spectral entropy indicates inhibitory activity
I_measure = np.mean(list(spectral_entropy_x.values()))

E_init = E_measure
I_init = I_measure

print(E_init)
print(I_init)


-0.04757563641953081
0.7114263594872726


In [ ]:
clustering techniques on EEG data to differentiate regions of high and low activity

In [16]:
print("Keys for PeakFrequencies_x:", PeakFrequencies_x.keys())
print("Keys for FastFourierTransform_x:", FastFourierTransform_x.keys())
print("Keys for SpectralCentroids_x:", SpectralCentroids_x.keys())
print("Keys for SpectralEdgeDensities_x:", SpectralEdgeDensities_x.keys())
print("Keys for spectral_entropy_x:", spectral_entropy_x.keys())
print("Keys for welchs_x:", welchs_x.keys())

Keys for PeakFrequencies_x: dict_keys(['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2'])
Keys for FastFourierTransform_x: dict_keys(['data_array'])
Keys for SpectralCentroids_x: dict_keys(['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2'])
Keys for SpectralEdgeDensities_x: dict_keys(['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2', 'Fpz'])
Keys for spectral_entropy_x: dict_keys(['data_array'])
Keys for welchs_x: dict_keys(['data_array'])


In [11]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Assume your data is loaded like this:
# BandPower_x, FastFourierTransform_x, PeakFrequencies_x, SpectralCentroids_x, SpectralEdgeDensities_x, spectral_entropy_x, welchs_x, mfdfa

# Stack the features horizontally
features = np.column_stack([FastFourierTransform_x, PeakFrequencies_x, SpectralCentroids_x, SpectralEdgeDensities_x, spectral_entropy_x, welchs_x])

# Standardize the features
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# Determine the optimal number of clusters using silhouette score
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
silhouette_scores = []

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters)
    cluster_labels = kmeans.fit_predict(features_standardized)
    silhouette_avg = silhouette_score(features_standardized, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Plot silhouette scores
plt.figure()
plt.bar(range_n_clusters, silhouette_scores)
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")
plt.show()

# Choose an optimal number of clusters and run k-means clustering
optimal_clusters = range_n_clusters[np.argmax(silhouette_scores)]
kmeans = KMeans(n_clusters=optimal_clusters)
clusters = kmeans.fit_predict(features_standardized)

# Visualize the clusters (you can choose any two features for a scatter plot or project them using PCA for visualization)
plt.figure(figsize=(10,7))
plt.scatter(features_standardized[:, 0], features_standardized[:, 1], c=clusters, cmap='rainbow')
plt.title("Clusters")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()


TypeError: float() argument must be a string or a real number, not 'dict'

In [ ]:
Derived Measures

In [ ]:
Source Localization

In [ ]:
dipole model or the current source density,  LORETA or sLORETA 

In [ ]:
# Initialize a PyCoBi object
pcb = pycobi()

# Use the Hodgkin-Huxley model as your system of ODEs
# Assuming the fortran version of the Hodgkin-Huxley model is available in a file named 'hodgkin_huxley.f'
pcb.load_equations('put model here.f')

# Set the initial conditions and parameters for your system
pcb.set_initial_conditions([V_init, 0.05, 0.6, 0.32])

# Define your control parameter and its range for the continuation
pcb.set_control_parameter('I', start=0, stop=100, step=1)

# Run the continuation
pcb.run_continuation()

# Display the bifurcation diagram
pcb.plot_bifurcation_diagram()

In [ ]:
Model-Based Mapping

In [ ]:
Jansen-Rit model